In [1]:
import numpy as np
import sparse
from tqdm import tqdm
from pyimzml.ImzMLParser import ImzMLParser
from spatialdata.models import Image2DModel

def collect_mz_intensity_data(imzml_file):
    """
    Collects m/z and intensity data from the imzML file, along with pixel coordinates.

    Returns:
        coords (list): List of coordinate pairs [row_indices, column_indices] for sparse COO.
        data (list): List of intensity values corresponding to the coordinates.
        unique_mz_values (numpy.ndarray): Sorted unique m/z values.
        pixel_coords (numpy.ndarray): Array of pixel coordinates.
    """
    parser = ImzMLParser(imzml_file)
    num_pixels = len(parser.coordinates)

    # Collect all m/z values to deduplicate
    mz_values_all = []
    pixel_coords = np.array(parser.coordinates)

    for idx in tqdm(range(num_pixels), desc="First Pass: Collecting m/z values"):
        mz_values, _ = parser.getspectrum(idx)
        mz_values_all.extend(mz_values)

    # Deduplicate and sort m/z values
    unique_mz_values = np.unique(mz_values_all)
    unique_mz_values = np.sort(unique_mz_values)

    # Create mapping from m/z values to indices for later use
    mz_index_map = {mz: i for i, mz in enumerate(unique_mz_values)}

    # Initialize lists to hold the coordinates and intensity data for COO array
    coords = [[], []]  # row indices, column indices
    data = []

    # Second pass to collect actual intensity data
    for idx in tqdm(range(num_pixels), desc="Second Pass: Collecting Intensity Data"):
        mz_values, intensities = parser.getspectrum(idx)
        for mz, intensity in zip(mz_values, intensities):
            column_index = mz_index_map.get(mz, -1)
            if column_index >= 0:
                coords[0].append(idx)  # row index (pixel index)
                coords[1].append(column_index)  # column index (m/z index)
                data.append(intensity)

    # Return the data required for constructing COO array
    return coords, data, unique_mz_values, pixel_coords


# Example usage
imzml_file = r"C:\Users\tvisv\OneDrive\Desktop\Taste of MSI\rsc Taste of MSI\Ingredient Classification MALDI\Original\20240605_pea_pos.imzML"
coords, data, unique_mz_values, pixel_coords = collect_mz_intensity_data(imzml_file)

# You now have the coordinates (coords), data (intensities), and the unique m/z values to create the COO array.
# You can then save `coords`, `data`, `unique_mz_values`, and `pixel_coords` to a file for later use.


c:\Users\tvisv\anaconda3\Lib\site-packages\pyimzml\ontology\ontology.py:92: UserWarning: Accession IMS:1000046 found with incorrect name "pixel size x". Updating name to "pixel size (x)".
  warn(
Second Pass: Collecting Intensity Data: 100%|██████████| 12737/12737 [00:43<00:00, 293.82it/s]


In [ ]:
max_x = max(pixel_coords[:, 0])
max_y = max(pixel_coords[:, 1])
max_c = len(unique_mz_values)
max_x, max_y
chunk_shape = (max_x, max_y)
chunk_shape

In [2]:
coords = np.array(coords)
data = np.array(data)

In [3]:
coo_array = sparse.COO(coords, data)
coo_array

Format,coo
Data Type,float32
Shape,"(12737, 331701)"
nnz,60149625
Density,0.014237016700143877
Read-only,True
Size,688.4M
Storage ratio,0.04


In [6]:
import zarr
import dask.array as da
import xarray as xr
from spatialdata.models import Image2DModel
import numpy as np

# Step 1: Open the Zarr array
# z = zarr.open(r"C:\Users\tvisv\OneDrive\Desktop\Taste of MSI\rsc Taste of MSI\Ingredient Classification MALDI\Original\20240505_onion pos.zarr", mode='r')

# Step 2: Create a Dask array from the Zarr array
# dask_array = da.from_zarr(z)
max_x = 271
max_y = 47
reshaped_array = coo_array.reshape((max_x, max_y, -1))
print(reshaped_array.shape)

# Optional: Inspect the Dask array
# print("Shape:", dask_array.shape)
# print("Chunks:", dask_array.chunks)

# Step 3: Create an xarray.DataArray from the Dask array
# Replace 'dim_0', 'dim_1', etc., with your actual dimension names
dimension_names = ['x', 'y', 'c']  # Modify based on your data's dimensions

data_array = xr.DataArray(
    reshaped_array,
    dims=dimension_names,
    # coords={'coord_name': coord_values},  # Optional: Add coordinates if needed
    name='my_data'  # Optional: Give a name to your DataArray
)

# Step 4: Use the parse method to create an Image2DModel
image_model = Image2DModel.parse(data_array)
image_model


(271, 47, 331701)
INFO     Transposing `data` of type: <class 'xarray.core.dataarray.DataArray'> to ('c', 'y', 'x').                 


<xarray.DataArray 'image' (c: 331701, y: 47, x: 271)> Size: 17GB
dask.array<transpose, shape=(331701, 47, 271), dtype=float32, chunksize=(2634, 47, 271), chunktype=sparse.COO>
Coordinates:
  * c        (c) int32 1MB 0 1 2 3 4 5 ... 331696 331697 331698 331699 331700
  * y        (y) float64 376B 0.5 1.5 2.5 3.5 4.5 ... 42.5 43.5 44.5 45.5 46.5
  * x        (x) float64 2kB 0.5 1.5 2.5 3.5 4.5 ... 267.5 268.5 269.5 270.5
Attributes:
    transform:  {'global': Identity }

In [ ]:
parsed_image = Image2DModel.parse(coo_array, dims=("y", "x"))